<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Regex_ChemCad_Import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import requests
import string
from collections import deque
import numpy as np

In [7]:
base_url = 'https://raw.githubusercontent.com/profteachkids/CHE2064/master/data/'
suffix = 'Props.txt'

comps = ['Isopropanol', 'Water']
N_comps = len(comps)
extract_single_props = {'Molecular Weight' : 'Mw',
                 'Critical Temperature' : 'Tc',
                 'Critical Pressure' : 'Pc',
                 'Critical Volume' : 'Vc',
                 'Acentric factor' : 'w',
                 'Normal boiling point' : 'Tb',
                 'Heat of vaporization' : 'Hvap'}
extract_coeff_props={'Vapor Pressure' : 'Pvap'}

single_props_pat = re.compile('^\s+([\w\s]+?)\s+:\s+([-.0-9e+]+)\s+[\w\s/]*$', re.MULTILINE)
coeffs_name_pat = re.compile("([\w ]+)\s[^\n]*?Equation.*?Coeffs:([- e\d.+]+)+?", re.DOTALL)
coeffs_pat = re.compile('([-\de.+]+)')

props_deque=deque()
for comp in comps:
  text = requests.get(base_url+comp + suffix).text
  single_props = dict(single_props_pat.findall(text))
  props={'Name': comp}
  for k,v in extract_single_props.items():
    props[v]=float(single_props.pop(k))

  coeffs_name_strings = dict(coeffs_name_pat.findall(text))
  for k,v in extract_coeff_props.items():
    coeffs = coeffs_pat.findall(coeffs_name_strings[k])
    for letter, value in zip(string.ascii_uppercase,coeffs):
      props[v+letter]=float(value)
  props_deque.append(props)

In [8]:
props={}
for prop in props_deque[0].keys():
  values = np.array([comp[prop] for comp in props_deque])
  props[prop]=values


In [9]:
props

{'Hvap': array([39832530., 40656800.]),
 'Mw': array([60.096, 18.015]),
 'Name': array(['Isopropanol', 'Water'], dtype='<U11'),
 'Pc': array([ 4764300., 22118230.]),
 'PvapA': array([92.935, 72.55 ]),
 'PvapB': array([-8177.1, -7206.7]),
 'PvapC': array([-10.031 ,  -7.1385]),
 'PvapD': array([3.9988e-06, 4.0460e-06]),
 'PvapE': array([2., 2.]),
 'PvapF': array([0., 0.]),
 'PvapG': array([0., 0.]),
 'Tb': array([355.41, 373.15]),
 'Tc': array([508.31, 647.35]),
 'Vc': array([0.2201  , 0.063494]),
 'w': array([0.6689, 0.348 ])}